# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
client = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(client.list_database_names())

['admin', 'config', 'local', 'travel_db', 'uk_food']


In [4]:
# Use the uk_food database and list collections
db = client['uk_food']
print(db.list_collection_names())

['establishments']


In [5]:
# Display one sample document from establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('683ef348ebe8cf6b9aa4133f'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


In [6]:
# Assign the establishments collection to a variable for further use
establishments = db['establishments']

In [7]:
# Confirm how many documents are in the establishments collection
print(establishments.count_documents({}))

38786


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

pprint(penang_flavours)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [10]:
# Insert the new restaurant into the establishments collection
result = establishments.insert_one(penang_flavours)

In [11]:
# Check that the new restaurant was inserted
print("Inserted document ID:", result.inserted_id)

Inserted document ID: 683f8936bbcdc401cb5511d9


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
btid_result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen", "BusinessTypeID": {"$ne": ""}},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
print(btid_result)
business_type_id = btid_result["BusinessTypeID"]

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the BusinessTypeID found
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)

In [14]:
# Confirm that the new restaurant was updated
print("Documents matched:", update_result.matched_count)
print("Documents updated:", update_result.modified_count)

Documents matched: 1
Documents updated: 0


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Establishments in Dover before delete:", dover_count)

Establishments in Dover before delete: 0


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print("Documents deleted:", delete_result.deleted_count)

Documents deleted: 0


In [17]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Establishments in Dover after delete:", dover_count_after)

Establishments in Dover after delete: 0


In [18]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()
pprint(sample_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('683ef348ebe8cf6b9aa4133f'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
update_lat = establishments.update_many(
    {"geocode.latitude": {"$type": "string"}},
    [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]
)
update_long = establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]
)
print("Latitude fields converted:", update_lat.modified_count)
print("Longitude fields converted:", update_long.modified_count)

Latitude fields converted: 1
Longitude fields converted: 1


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
update_rating = establishments.update_many(
    {"RatingValue": {"$type": "string", "$regex": "^[1-5]$"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)
print("RatingValue fields converted:", update_rating.modified_count)

RatingValue fields converted: 0


In [22]:
# Check that the coordinates and rating value are now numbers
sample = establishments.find_one(
    {"RatingValue": {"$type": "int"}, "geocode.latitude": {"$type": "double"}, "geocode.longitude": {"$type": "double"}}
)
pprint(sample)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('683ef348ebe8cf6b9aa4133f'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,
